In [106]:
# Attempt 1: Dropping more columns, reduce epochs to 50

In [124]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import warnings 
warnings.filterwarnings('ignore')
import os
from tensorflow.keras.callbacks import ModelCheckpoint

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [125]:
# Drop the non-beneficial ID columns
application_df = application_df.drop(columns=["EIN", "NAME", "INCOME_AMT" ], axis=1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,N,142590,1


In [126]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [127]:
# Look at APPLICATION_TYPE value counts for binning
app_counts = application_df.APPLICATION_TYPE.value_counts()
app_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [128]:
# Determine which values to replace if counts are less than ...?
replace_application = list(app_counts[app_counts < 500].index)


# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [129]:
# Look at CLASSIFICATION value counts for binning
classification_value_counts = application_df['CLASSIFICATION'].value_counts()
classification_value_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [130]:
# Determine which values to replace if counts are less than ..?
replace_class = list(classification_value_counts[classification_value_counts < 500].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [131]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'SPECIAL_CONSIDERATIONS']

In [132]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


In [133]:
application_df = application_df.merge(encode_df, left_index=True,right_index=True)
application_df = application_df.drop(columns = application_cat)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


In [134]:
# Split our preprocessed data into our features and target arrays
y = application_df.IS_SUCCESSFUL
X = application_df.drop("IS_SUCCESSFUL", axis=1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify=y)

In [135]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
X_train_scaled.shape

(25724, 35)

In [136]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
nodes_hidden_layer1 = 80
nodes_hidden_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_hidden_layer1, activation="relu", input_dim=number_input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_hidden_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_44 (Dense)            (None, 80)                2880      
                                                                 
 dense_45 (Dense)            (None, 30)                2430      
                                                                 
 dense_46 (Dense)            (None, 1)                 31        
                                                                 
Total params: 5,341
Trainable params: 5,341
Non-trainable params: 0
_________________________________________________________________


In [137]:
# Compile the model
os.makedirs("checkpoints/", exist_ok=True)
checkpoint_path="checkpoints/weights.{epoch:02d}.hdf5"

nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [138]:
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50, callbacks=[cp_callback])

Epoch 1/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5731 - accuracy: 0.7203
Epoch 2/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5628 - accuracy: 0.7240
Epoch 3/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5604 - accuracy: 0.7243
Epoch 4/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5587 - accuracy: 0.7259
Epoch 5/50
790/804 [============================>.] - ETA: 0s - loss: 0.5576 - accuracy: 0.7265
Epoch 5: saving model to checkpoints\weights.05.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5582 - accuracy: 0.7262
Epoch 6/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5574 - accuracy: 0.7271
Epoch 7/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5566 - accuracy: 0.7271
Epoch 8/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5563 - accuracy: 0.7271
Epoch 9/50
804/804 [==============================] - 2s 2ms/s

In [139]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5567 - accuracy: 0.7311 - 533ms/epoch - 2ms/step
Loss: 0.5566686391830444, Accuracy: 0.7310787439346313


In [140]:
# Save & Export results to HDF5 file named AlphabetSoupCharity_Optimization.h5
nn.save("AlphabetSoupCharity_Optimization1.h5")

In [141]:
# Attempt 2: Adding more hidden layers and more neurons

In [149]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
nodes_hidden_layer1 = 80
nodes_hidden_layer2 = 30
nodes_hidden_layer3 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_hidden_layer1, activation="relu", input_dim=number_input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_hidden_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_hidden_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_55 (Dense)            (None, 80)                2880      
                                                                 
 dense_56 (Dense)            (None, 30)                2430      
                                                                 
 dense_57 (Dense)            (None, 10)                310       
                                                                 
 dense_58 (Dense)            (None, 1)                 11        
                                                                 
Total params: 5,631
Trainable params: 5,631
Non-trainable params: 0
_________________________________________________________________


In [150]:
# Compile the model
os.makedirs("checkpoints/", exist_ok=True)
checkpoint_path="checkpoints/weights.{epoch:02d}.hdf5"

nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [151]:
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5739 - accuracy: 0.7192
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5610 - accuracy: 0.7252
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5603 - accuracy: 0.7251
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5581 - accuracy: 0.7265
Epoch 5/100
788/804 [============================>.] - ETA: 0s - loss: 0.5583 - accuracy: 0.7258
Epoch 5: saving model to checkpoints\weights.05.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5576 - accuracy: 0.7266
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5574 - accuracy: 0.7253
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5564 - accuracy: 0.7274
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5563 - accuracy: 0.7286
Epoch 9/100
804/804 [==============================] -

804/804 [==============================] - 2s 2ms/step - loss: 0.5470 - accuracy: 0.7324
Epoch 64/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5473 - accuracy: 0.7320
Epoch 65/100
786/804 [============================>.] - ETA: 0s - loss: 0.5472 - accuracy: 0.7337
Epoch 65: saving model to checkpoints\weights.65.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5476 - accuracy: 0.7332
Epoch 66/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5473 - accuracy: 0.7324
Epoch 67/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5472 - accuracy: 0.7324
Epoch 68/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5470 - accuracy: 0.7329
Epoch 69/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5472 - accuracy: 0.7332
Epoch 70/100
790/804 [============================>.] - ETA: 0s - loss: 0.5462 - accuracy: 0.7334
Epoch 70: saving model to checkpoints\weights.70.hdf5
804/804 

In [152]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5586 - accuracy: 0.7294 - 555ms/epoch - 2ms/step
Loss: 0.5585982799530029, Accuracy: 0.7294460535049438


In [153]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization2.h5")

In [154]:
# Attempt 3: Using different activation functions for hidden layers

In [167]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
nodes_hidden_layer1 = 80
nodes_hidden_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_hidden_layer1, activation="elu", input_dim=number_input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_hidden_layer2, activation="elu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_69 (Dense)            (None, 80)                2880      
                                                                 
 dense_70 (Dense)            (None, 30)                2430      
                                                                 
 dense_71 (Dense)            (None, 1)                 31        
                                                                 
Total params: 5,341
Trainable params: 5,341
Non-trainable params: 0
_________________________________________________________________


In [168]:
# Compile the model
os.makedirs("checkpoints/", exist_ok=True)
checkpoint_path="checkpoints/weights.{epoch:02d}.hdf5"

nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [169]:
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5809 - accuracy: 0.7145
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5663 - accuracy: 0.7214
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5627 - accuracy: 0.7242
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5620 - accuracy: 0.7260
Epoch 5/100
781/804 [============================>.] - ETA: 0s - loss: 0.5609 - accuracy: 0.7251
Epoch 5: saving model to checkpoints\weights.05.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5615 - accuracy: 0.7250
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5597 - accuracy: 0.7253
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5598 - accuracy: 0.7256
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5585 - accuracy: 0.7273
Epoch 9/100
804/804 [==============================] -

804/804 [==============================] - 2s 2ms/step - loss: 0.5500 - accuracy: 0.7310
Epoch 64/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5501 - accuracy: 0.7317
Epoch 65/100
791/804 [============================>.] - ETA: 0s - loss: 0.5494 - accuracy: 0.7322
Epoch 65: saving model to checkpoints\weights.65.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5497 - accuracy: 0.7319
Epoch 66/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5497 - accuracy: 0.7310
Epoch 67/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5496 - accuracy: 0.7317
Epoch 68/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5497 - accuracy: 0.7303
Epoch 69/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5497 - accuracy: 0.7319
Epoch 70/100
784/804 [============================>.] - ETA: 0s - loss: 0.5504 - accuracy: 0.7312
Epoch 70: saving model to checkpoints\weights.70.hdf5
804/804 

In [170]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5549 - accuracy: 0.7283 - 557ms/epoch - 2ms/step
Loss: 0.5549084544181824, Accuracy: 0.7282798886299133


In [171]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")